In [14]:
import fastplotlib as fpl
import os
import sys
import masknmf
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%matplotlib inline

In [2]:
data = tifffile.imread("../../demo_data/demoMovie.tif")

# Define a rigid motion correction strategy. Optionally provide a template as a 2D torch.Tensor

In [4]:
max_rigid_shifts = [3, 3]
template = None
rigid_strategy = masknmf.RigidMotionCorrector(max_rigid_shifts, template = template)

# Run the template estimator

In [12]:
rigid_strategy.compute_template(data,
                                device="cuda")

# Define a motion correction array, which lazily loads motion corrected frames aligned via the registation_strategy

In [6]:
moco_results = masknmf.RegistrationArray(data,
                                         rigid_strategy,
                                         device = "cuda")

# How to access shifts (and corresponding motion corrected frames). 

In [8]:
moco_shifts = moco_results.shifts[0:500] #Shift data for first 500 frames

In [13]:
fig, ax = plt.subplots(1, 2)
ax[0].plot(moco_shifts[:, 0])
ax[0].set_title("vertical shifts")
ax[1].plot(moco_shifts[:, 1])
ax[1].set_title("horizontal shifts")
plt.show()

# Visualize the raw vs. registered data via fastplotlib

In [11]:
iw = fpl.ImageWidget(data = [data, moco_results, rigid_strategy.template.cpu().numpy()],
                    names = ['raw', 'motion corrected', 'template'],
                    figure_shape = (1, 3))
iw.cmap = "gray"
iw.show()